In [4]:
import google.auth
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json
import tempfile
import pandas as pd
import openpyxl
from datetime import datetime, timedelta
import logging

# Set up logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

def print_service_account_info(creds):
    """Print service account details for verification"""
    try:
        logging.info("=== Service Account Information ===")
        logging.info(f"Service Account Email: {creds.service_account_email}")
        logging.info(f"Project ID: {creds.project_id}")
        logging.info("=====================================")
    except Exception as e:
        logging.error(f"Could not get service account details: {str(e)}")

def setup_credentials():
    """Set up Google credentials with error handling"""
    try:
        credentials_path = r'C:\Users\IAN NAMBOGA MADETE\Downloads\boxwood-chassis-438812-q4-da810f36cfed.json'
        logging.info(f"Reading credentials from: {credentials_path}")
        
        with open(credentials_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        with tempfile.NamedTemporaryFile(mode='w', encoding='utf-8', delete=False) as temp_file:
            temp_file.write(json.dumps(data))
            logging.info(f"Temporary credentials file created at: {temp_file.name}")
            
        creds = service_account.Credentials.from_service_account_file(
            temp_file.name, 
            scopes=['https://www.googleapis.com/auth/webmasters.readonly']
        )
        logging.info("Credentials successfully created")
        return creds
    except FileNotFoundError:
        logging.error(f"Credentials file not found at: {credentials_path}")
        raise
    except json.JSONDecodeError:
        logging.error("Invalid JSON in credentials file")
        raise
    except Exception as e:
        logging.error(f"Error setting up credentials: {str(e)}")
        raise

def get_search_console_service():
    """Build and return the Search Console service"""
    try:
        creds = setup_credentials()
        service = build('searchconsole', 'v1', credentials=creds)
        logging.info("Search Console service successfully created")
        return service
    except HttpError as error:
        logging.error(f'HTTP error occurred: {error}')
        raise
    except Exception as e:
        logging.error(f'Error creating service: {str(e)}')
        raise
        
def get_site_list(service):
    """Get list of sites in Search Console"""
    try:
        sites = service.sites().list().execute()
        logging.info(f"Found {len(sites.get('siteEntry', []))} sites")
        return sites
    except HttpError as error:
        logging.error(f'Error fetching sites: {error}')
        raise
    except Exception as e:
        logging.error(f'Unexpected error fetching sites: {str(e)}')
        raise

def get_leading_queries(service, site_url):
    """Get leading queries for each URL"""
    try:
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=28)).strftime('%Y-%m-%d')
        logging.info(f"Fetching data for period: {start_date} to {end_date}")
        
        request = {
            'startDate': start_date,
            'endDate': end_date,
            'dimensions': ['page', 'query'],
            'rowLimit': 25000,
            'startRow': 0
        }
        
        logging.info(f"Requesting data for site: {site_url}")
        response = service.searchanalytics().query(siteUrl=site_url, body=request).execute()
        
        if not response.get('rows'):
            logging.warning("No data rows returned from Search Console")
            return []
            
        logging.info(f"Received {len(response['rows'])} rows of data")
        
        # Process the data as before...
        results = []
        url_data = {}
        
        for row in response['rows']:
            url = row['keys'][0]
            query = row['keys'][1]
            clicks = row.get('clicks', 0)
            impressions = row.get('impressions', 0)
            
            if url not in url_data:
                url_data[url] = {
                    'queries_clicks': [],
                    'queries_impressions': []
                }
            
            url_data[url]['queries_clicks'].append((query, clicks))
            url_data[url]['queries_impressions'].append((query, impressions))
        
        for url, data in url_data.items():
            clicks_sorted = sorted(data['queries_clicks'], 
                                key=lambda x: x[1], 
                                reverse=True)
            impressions_sorted = sorted(data['queries_impressions'], 
                                     key=lambda x: x[1], 
                                     reverse=True)
            
            results.append({
                'URL': url,
                'Leading Query (Clicks)': clicks_sorted[0][0] if clicks_sorted and clicks_sorted[0][1] > 0 else "-",
                'Leading Query (Impressions)': impressions_sorted[0][0] if impressions_sorted else "-"
            })
        
        logging.info(f"Processed {len(results)} URLs with their leading queries")
        return results
        
    except HttpError as error:
        logging.error(f'Error fetching query data: {error}')
        raise
    except Exception as e:
        logging.error(f'Unexpected error processing queries: {str(e)}')
        raise

def export_to_excel(results, filename):
    """Export results to Excel file"""
    try:
        df = pd.DataFrame(results)
        df.to_excel(filename, index=False)
        logging.info(f"Data successfully exported to {filename}")
    except Exception as error:
        logging.error(f'Error exporting to Excel: {error}')
        raise

def main():
    try:
        logging.info("Starting Search Console data extraction")
        
        service = get_search_console_service()
        sites_data = get_site_list(service)
        
        if not sites_data.get('siteEntry'):
            logging.warning("No sites found in Search Console")
            print("No sites found in your Search Console account.")
            return
        
        print("\nAvailable sites:")
        for site in sites_data['siteEntry']:
            print(f"- {site['siteUrl']}")
        
        selected_site = input("\nEnter the complete site URL from above: ").strip()
        
        if not any(site['siteUrl'] == selected_site for site in sites_data['siteEntry']):
            logging.error(f"Invalid site URL selected: {selected_site}")
            print("Error: Selected site URL does not match any available sites.")
            return
        
        results = get_leading_queries(service, selected_site)
        
        if not results:
            print("No data found for the selected site.")
            return
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'leading_queries_{timestamp}.xlsx'
        export_to_excel(results, filename)
        
        print(f"\nProcessed {len(results)} URLs")
        print(f"Results exported to: {filename}")
        
    except Exception as e:
        logging.error(f"Main execution failed: {str(e)}")
        print(f"\nAn error occurred: {str(e)}")
        print("Check the logs for more details.")

if __name__ == "__main__":
    main()

2024-10-19 13:33:20,367 - INFO - Starting Search Console data extraction
2024-10-19 13:33:20,367 - INFO - Reading credentials from: C:\Users\IAN NAMBOGA MADETE\Downloads\boxwood-chassis-438812-q4-da810f36cfed.json
2024-10-19 13:33:20,367 - INFO - Temporary credentials file created at: C:\Users\IANNAM~1\AppData\Local\Temp\tmpmmwuplmi
2024-10-19 13:33:20,411 - INFO - Credentials successfully created
2024-10-19 13:33:20,416 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-10-19 13:33:20,417 - INFO - Search Console service successfully created
2024-10-19 13:33:22,363 - INFO - Found 0 sites
2024-10-19 13:33:22,363 - WARNING - No sites found in Search Console


No sites found in your Search Console account.
